In [1]:
import pandas as pd
import networkx as nx

from program.flow import Flow
from program.graph_construction import Create_graph, Create_Graph_old

# PROTOTYPE

### Objectives

- Extract data from csv files to put in an appropriate data structure
- Build a network and find shortest path

In [2]:
"""Loading information about streams and topology of a network"""

stream_file = "test_cases/small-streams.v2.csv"
topology_file = "test_cases/small-topology.v2.csv"
streams = pd.read_csv(stream_file,
                      names=['PCP','StreamName','StreamType','SourceNode','DestinationNode','Size','Period','Deadline']
                     )

topology_cols = [str(i) for i in range(7)]
topology = pd.read_csv(topology_file,names=topology_cols).groupby('0')

In [3]:
"""Extracting the data about topology of network"""

switches = topology.get_group('SW')
switches = switches.drop(columns=['4','5','6'])
switches.columns = ['DeviceType','DeviceName','Ports','Domain']

end_systems = topology.get_group('ES')
end_systems = end_systems.drop(columns=['4','5','6'])
end_systems.columns = ['DeviceType','DeviceName','Ports','Domain']

links = topology.get_group('LINK')
links.columns = ['LINK','LinkID','SourceDevice','SourcePort','DestinationDevice','DestinationPort','Domain']

In [5]:
G = Create_Graph_old(end_systems,switches,links)
G = Create_graph(pd.concat([end_systems,switches]).drop(columns='Domain'),links.drop(columns='Domain'))

In [14]:
streams.head()

,PCP,StreamName,StreamType,SourceNode,DestinationNode,Size,Period,Deadline
0,7,VLAN_0_Flow_0,ATS,node0_0_1_0,node0_0_3_0,100,10000,15000
1,7,VLAN_0_Flow_1,ATS,node0_0_1_1,node0_0_3_1,100,10000,15000
2,7,VLAN_0_Flow_2,ATS,node0_0_1_0,node0_0_3_1,100,10000,15000
3,7,VLAN_0_Flow_3,ATS,node0_0_1_1,node0_0_3_0,100,10000,15000
4,7,VLAN_0_Flow_4,ATS,node0_0_1_0,node0_0_3_0,100,10000,15000


In [22]:
test = Flow(stream)

In [23]:
test

AttributeError: 'Flow' object has no attribute 'path'

In [16]:
flows = []
for _, stream in streams.iterrows():
    flow = Flow(stream)
    flow.find_path(G)
    flows.append(flow)

for flow in flows:
    flow.fill_nodes()
    pass

ValueError: too many values to unpack (expected 2)

In [13]:
for flow in flows:
    flow.get_total_delay()
    print(flow)

VLAN_0_Flow_0,18.4,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_1,18.4,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_2,18.4,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_3,18.4,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_4,18.4,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_5,18.4,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_6,18.4,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_7,18.4,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_8,18.4,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_9,18.4,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_10,32.0,18000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_11,32.0,18000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_12,32.0,18000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_13,32.0,18000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_14,32.0,18000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_

In [14]:
for flow in flows:
    flow.new_get_total_delay()
    print(flow)

AttributeError: 'Flow' object has no attribute 'new_get_total_delay'

In [ ]:
flows[0].get_total_delay()
    # print(f"{flow.name}, {flow.total_delay * 1e6}")
print(flows[0])

Stream_1,70.12,9389,ES_1->Switch_1->Switch_8->Switch_7->ES_14


In [ ]:
for k, item in G.edges.items():
    print(item)

{'source_port': 1.0, 'destination_port': 1.0, 'link_id': 'Link_1', 'flows': {7: [Stream_1,70.12,9389,ES_1->Switch_1->Switch_8->Switch_7->ES_14, Stream_2,92.424,9186,ES_1->Switch_1->Switch_2->Switch_3->Switch_4->ES_8], 2: [Stream_3,129.528,9867,ES_1->Switch_1->Switch_8->Switch_7->Switch_6->ES_12, Stream_10,27.024,1673,ES_4->Switch_2->Switch_1->ES_1]}}
{'source_port': 1.0, 'destination_port': 2.0, 'link_id': 'Link_2', 'flows': {7: [Stream_4,49.2,4052,ES_2->Switch_1->Switch_8->ES_15], 4: [Stream_5,54.944,7762,ES_2->Switch_1->Switch_8->ES_16], 1: [Stream_6,89.792,5706,ES_2->Switch_1->Switch_8->Switch_7->ES_13], 2: [Stream_37,57.04,6452,ES_13->Switch_7->Switch_8->Switch_1->ES_2]}}
{'source_port': 3.0, 'destination_port': 3.0, 'link_id': 'Link_17', 'flows': {7: [Stream_2,92.424,9186,ES_1->Switch_1->Switch_2->Switch_3->Switch_4->ES_8, Stream_45,55.768,1007,ES_15->Switch_8->Switch_1->Switch_2->ES_4], 2: [Stream_10,27.024,1673,ES_4->Switch_2->Switch_1->ES_1], 0: [Stream_17,100.896,7943,ES_6->Sw